In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [6]:
    !pip install -q git+https://github.com/huggingface/transformers

In [7]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
def simple_prompt(messages, tokenizer, model):
    model_inputs = tokenizer.apply_chat_template(messages,return_tensors = "pt")
#     print(model_inputs)
    model_inputs = model_inputs.to('cuda')

    generated_ids = model.generate(
        model_inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens = 1000,
        do_sample = True,
    )

    decoded = tokenizer.batch_decode(generated_ids)

#     print(decoded[0])
    return decoded[0]

In [9]:
import time
def simple_chat(tokenizer, model):
    user_input = None
    immediate_context = []
    while user_input != "bye":
        message = {"role": "user", "content": f"{user_input}"}
        
        if user_input == None:
            print('------------------------------------------------------------------------------------------------------')
            user_input = input("| You |\n-------\n")
            continue
        immediate_context.append(message)
#         print(immediate_context)
                
        res = simple_prompt(immediate_context, tokenizer, model)
#         print(res)
        
        answer = res.split('[/INST]')[-1].strip('</s>').strip()
        
        print('------------------------------------------------------------------------------------------------------')
        padding = ' ' * 91
        print(padding, " | Model |\n", padding, "---------\n", answer)
        
        response = {"role":"assistant", "content": f"{answer}"}
        immediate_context.append(response)
        
        print('------------------------------------------------------------------------------------------------------')
        user_input = input("| You |\n-------\n")

In [70]:
simple_chat(tokenizer, model)

------------------------------------------------------------------------------------------------------


| You |
-------
 quit


------------------------------------------------------------------------------------------------------
                                                                                             | Model |
                                                                                             ---------
 Hello! It was nice to meet you. Don't hesitate to reach out if you need more help or if there's anything else I can assist you with. Have a great day!
------------------------------------------------------------------------------------------------------


| You |
-------
 bye


In [12]:
import signal

# Handler function for the alarm signal
def alarm_handler(signum, frame):
    print("\nNo input received within 5 seconds. Exiting...")
    exit()

# Set the alarm signal handler
signal.signal(signal.SIGALRM, alarm_handler)

print("Please input something within 5 seconds:")
signal.alarm(5)  # Set the alarm for 5 seconds

try:
    user_input = input()
    signal.alarm(0)  # Cancel the alarm if input is received
    print("You entered:", user_input)
except KeyboardInterrupt:
    pass  # If the user manually interrupts (Ctrl+C), handle it gracefully


Please input something within 5 seconds:

No input received within 5 seconds. Exiting...
